<a href="https://colab.research.google.com/github/MHHamdan/Artificial-Intelligence-Engineer/blob/Data-Science-with-Python/Project_1_Classify_hate_vs_Nonhate_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Unidecode


     |████████████████████████████████| 245kB 5.3MB/s 


In [ ]:
# Import the necessary packages
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt 
import seaborn as sb
from nltk.corpus import stopwords
import warnings 
warnings.filterwarnings("ignore")
import unidecode
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import matplotlib.animation as animation
import operator
import plotly.express as px
from collections import Counter
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Load the tweets file using read_csv function from Pandas package. 



In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/AI_ML_Engineer/Projects/Natural Language Processing NLP/Project_1_Tweet_hate_classification/TwitterHate.csv')

In [ ]:
dataset.shape # present the shape of the dataset

(31962, 3)

In [ ]:
dataset.head() # show the first 5 records 

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
dataset.tail() # show the last 5 records 

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [ ]:
dataset.columns # get columns

Index(['id', 'label', 'tweet'], dtype='object')

In [ ]:
dataset.describe() # describe the dataset

,id,label
count,31962.000000,31962.000000
mean,15981.500000,0.070146
std,9226.778988,0.255397
min,1.000000,0.000000
25%,7991.250000,0.000000
50%,15981.500000,0.000000
75%,23971.750000,0.000000
max,31962.000000,1.000000


#Get the tweets into a list for easy text cleanup and manipulation.



In [ ]:
tweet_list = dataset['tweet'].tolist()

print(f"\ntweet_list_list:\n{tweet_list}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(f"\ntweet_list_list:\ntype:{type(tweet_list)}")


tweet_list_list:
type:<class 'list'>


In [ ]:
tweet_list[0:10]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation',
 '[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo  ',
 ' @user camping tomorrow @user @user @user @user @user @user @user dannyâ\x80¦',
 "the next school year is the year for exams.ð\x9f\x98¯ can't think about that ð\x9f\x98\xad #school #exams   #hate #imagine #actorslife #revolutionschool #girl",
 'we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â\x80¦ ',
 " @user @user welcome here !  i'm   it's so #gr8 ! "]

#To cleanup: 



##1. Normalize the casing.



In [ ]:
tweet_list_lower_case = (list(map(lambda x: x.lower(), tweet_list)))

In [ ]:
tweet_list_lower_case[0:10]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation',
 '[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo  ',
 ' @user camping tomorrow @user @user @user @user @user @user @user dannyâ\x80¦',
 "the next school year is the year for exams.ð\x9f\x98¯ can't think about that ð\x9f\x98\xad #school #exams   #hate #imagine #actorslife #revolutionschool #girl",
 'we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â\x80¦ ',
 " @user @user welcome here !  i'm   it's so #gr8 ! "]

##2. Using regular expressions, remove user handles. These begin with '@’.


In [ ]:
import re,string

def strip_links(tweet):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, tweet)
    for link in links:
        tweet = tweet.replace(link[0], ', ')    
    return tweet

In [ ]:
def strip_all_entities(tweet):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            tweet = tweet.replace(separator,' ')
    words = []
    for word in tweet.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [ ]:
tweet_list_no_hash = []
for t in tweet_list_lower_case:
    tweet_list_no_hash.append(strip_all_entities(strip_links(t)))

In [ ]:
tweet_list_no_hash[0:10]

['when a father is dysfunctional and is so selfish he drags his kids into his dysfunction',
 'thanks for credit i can t use cause they don t offer wheelchair vans in pdx',
 'bihday your majesty',
 'i love u take with u all the time in urð\x9f\x93± ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91 ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦',
 'factsguide society now',
 '2 2 huge fan fare and big talking before they leave chaos and pay disputes when they get there',
 'camping tomorrow dannyâ\x80¦',
 'the next school year is the year for exams ð\x9f\x98¯ can t think about that ð\x9f\x98\xad',
 'we won love the land â\x80¦',
 'welcome here i m it s so']

##3. Using regular expressions, remove URLs.



In [ ]:
tweets_nourl = [re.sub("\w+://\S+","", twt) for twt in tweet_list_no_hash]

In [ ]:
tweets_nourl[0:10]

['when a father is dysfunctional and is so selfish he drags his kids into his dysfunction',
 'thanks for credit i can t use cause they don t offer wheelchair vans in pdx',
 'bihday your majesty',
 'i love u take with u all the time in urð\x9f\x93± ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91 ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦',
 'factsguide society now',
 '2 2 huge fan fare and big talking before they leave chaos and pay disputes when they get there',
 'camping tomorrow dannyâ\x80¦',
 'the next school year is the year for exams ð\x9f\x98¯ can t think about that ð\x9f\x98\xad',
 'we won love the land â\x80¦',
 'welcome here i m it s so']

In [ ]:
len(tweets_nourl)

31962

##4. Using TweetTokenizer from NLTK, tokenize the tweets into individual terms.

In [ ]:
from nltk.tokenize import TweetTokenizer
tkn = TweetTokenizer()


In [ ]:
tweet_token = [tkn.tokenize(sent) for sent in tweets_nourl]
print(tweet_token[0])


['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction']


In [ ]:
print(len(tweet_token))


31962


##5. Removing punctuation, stop words, redundant terms like ‘rt’, ‘amp’, and removing terms with length of 1:



In [ ]:
from nltk.corpus import stopwords
from string import punctuation

stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)


##6. Remove redundant terms like ‘amp’, ‘rt’, etc.



In [ ]:
#Adding some specific punctuation from the  data :
stop_punct.extend(['...','``',"''",".."])
stop_context = ['rt', 'amp']
#Final stop word list including all of these:
stop_final = stop_nltk + stop_punct + stop_context


##7.Remove ‘#’ symbols from the tweet while retaining the term.

#Extra cleanup by removing terms with a length of 1.



In [ ]:
def del_stop(sent):
    return [re.sub("#","",term) for term in sent if ((term not in stop_final) & (len(term)>1))]


In [ ]:
#Applying the function on the data:
tweets_clean = [del_stop(tweet) for tweet in tweet_token]


In [ ]:
print(tweets_clean[0])
print(len(tweets_clean))

['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction']
31962


#Check out the top terms in the tweets:

1. First, get all the tokenized terms into one large list.



In [ ]:
term_list = []
for tweet in tweets_clean:
    term_list.extend(tweet)


In [ ]:
print(term_list)
print(len(term_list))
print(type(term_list))

['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'thanks', 'credit', 'use', 'cause', 'offer', 'wheelchair', 'vans', 'pdx', 'bihday', 'majesty', 'love', 'take', 'time', 'urð', 'factsguide', 'society', 'huge', 'fan', 'fare', 'big', 'talking', 'leave', 'chaos', 'pay', 'disputes', 'get', 'camping', 'tomorrow', 'dannyâ', 'next', 'school', 'year', 'year', 'exams', 'think', 'love', 'land', 'welcome', 'consumer', 'price', 'index', 'mom', 'climbed', 'previous', 'may', 'selfish', 'get', 'see', 'daddy', 'today', 'calls', 'middle', 'school', 'build', 'wall', 'chant', 'comment', 'ouch', 'junior', 'angryð', 'got7', 'thankful', 'paner', 'retweet', 'agree', 'smiles', 'around', 'via', 'ig', 'user', 'make', 'people', 'know', 'essential', 'oils', 'made', 'chemicals', 'people', 'blaming', 'ha', 'conceded', 'goal', 'fat', 'rooney', 'gave', 'away', 'free', 'kick', 'knowing', 'bale', 'hit', 'sad', 'little', 'dude', 'product', 'day', 'happy', 'man', 'tool', 'time', 'open', 'drink', 'lump

2. Use the counter and find the 10 most common terms.

In [ ]:
from collections import Counter
res = Counter(term_list)
res.most_common(10)


[('day', 2063),
 ('happy', 1569),
 ('love', 1222),
 ('time', 1094),
 ('today', 1007),
 ('like', 977),
 ('new', 928),
 ('get', 917),
 ('people', 813),
 ('good', 795)]

#Data formatting for predictive modeling:

##1.Join the tokens back to form strings. This will be required for the vectorizers.



In [ ]:
tweets_clean = [" ".join(tweet) for tweet in tweets_clean]

In [ ]:
print(len(tweets_clean))

31962


##2. Assign x and y.



In [ ]:
X = tweets_clean
y = dataset.label.values

In [ ]:
len(X), len(y)

(31962, 31962)

##3. Perform train_test_split using sklearn.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)


#7,We’ll use TF-IDF values for the terms as a feature to get into a vector space model.



##1. Import TF-IDF  vectorizer from sklearn.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


##2.Instantiate with a maximum of 5000 terms in your vocabulary.



In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000)


##3. Fit and apply on the train set.



In [ ]:
X_train_bow = vectorizer.fit_transform(X_train)


##3.Apply on the test set.

In [ ]:
X_test_bow = vectorizer.transform(X_test)


In [ ]:
X_train_bow.shape, X_test_bow.shape


((22373, 5000), (9589, 5000))

#8.Model building: Ordinary Logistic Regression



##1.Instantiate Logistic Regression from sklearn with default parameters.



In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


##2.Fit into  the train data.



In [ ]:
logreg.fit(X_train_bow, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

##3.Make predictions for the train and the test set.

In [ ]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)


#9.Model evaluation: Accuracy, recall, and f_1 score.



##1.Report the accuracy on the train set.



In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_train, y_train_pred)


0.9469002815894159

##2.Report the recall on the train set: decent, high, or low.



##3.Get the f1 score on the train set.

In [ ]:
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97     20815
           1       0.95      0.25      0.40      1558

    accuracy                           0.95     22373
   macro avg       0.95      0.63      0.68     22373
weighted avg       0.95      0.95      0.93     22373



#10.Looks like you need to adjust the class imbalance, as the model seems to focus on the 0s.



##1.Adjust the appropriate class in the LogisticRegression model.

In [ ]:
logreg = LogisticRegression(class_weight="balanced")


#11.Train again with the adjustment and evaluate.



##1.Train the model on the train set.



In [ ]:
logreg.fit(X_train_bow, y_train)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#2.Evaluate the predictions on the train set: accuracy, recall, and f_1 score.

In [ ]:
y_train_pred = logreg.predict(X_train_bow)
accuracy_score(y_train, y_train_pred)


0.9292897689178921

In [ ]:
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

           0       1.00      0.93      0.96     20815
           1       0.50      0.95      0.65      1558

    accuracy                           0.93     22373
   macro avg       0.75      0.94      0.81     22373
weighted avg       0.96      0.93      0.94     22373



So much better on the train set! A recall of 97%! Looks like the parameter improved  a lot.  
The f1_score is also better at 0.74. 


#12.Regularization and Hyperparameter tuning:



#1.Import GridSearch and StratifiedKFold because of class imbalance.



In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold


##2.Provide the parameter grid to choose for ‘C’ and ‘penalty’ parameters.



In [ ]:
#Create the parameter grid based on the results of random search. 

param_grid = {
    'C': [0.01,0.1,1,10,100],
    'penalty': ["l1","l2"]
    }


##3.Use a balanced class weight while instantiating the logistic regression.

In [ ]:
#Instantiating the logistic regression model with a balanced class weight:
classifier_lr = LogisticRegression(class_weight="balanced")


#13.Find the parameters with the best recall in cross validation.



Supply stratified k-fold as out cv strategy to GridSearchCV. To stratify as there is heavy class imbalance in the dataset.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

##1.Choose ‘recall’ as the metric for scoring.



##2.Choose stratified 4 fold cross validation scheme.



In [ ]:
grid_search = GridSearchCV(estimator = classifier_lr, param_grid = param_grid, 
                          cv = StratifiedKFold(4), n_jobs = -1, verbose = 1, scoring = "recall" )


##3.Fit into  the train set.

In [ ]:
grid_search.fit(X_train_bow, y_train)


Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    4.1s finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=1)

#14.What are the best parameters?


From cross validation, the best parameters are: C = 1, penalty = “l2”.


#15.Predict and evaluate using the best estimator.



##1.Use the best estimator from the grid search to make predictions on the test set.



In [ ]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)


#2.What is the recall on the test set for the toxic comments?



In [ ]:
print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.97      0.91      0.94      8905
           1       0.37      0.68      0.48       684

    accuracy                           0.89      9589
   macro avg       0.67      0.80      0.71      9589
weighted avg       0.93      0.89      0.91      9589



##3.What is the f_1 score?

The model did a good job on the test set. The f1_score for 1 class is 0.48 and the recall is 0.68. 